## Ejercicio 1: Análisis de Datos de Calidad del Aire en California

Utilizando el dataset de calidad del aire de California disponible en:
https://www.kaggle.com/datasets/sogun3/uspollution

O directamente del EPA (Environmental Protection Agency):
https://aqs.epa.gov/aqsweb/airdata/download_files.html

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración básica de visualización
plt.style.use('default')  # Usando el estilo por defecto en lugar de seaborn
sns.set_theme()  # Configuración moderna de seaborn
pd.set_option('display.max_columns', None)

### 1. Carga y filtrado inicial de datos

Carga el dataset en un DataFrame y filtra los datos para quedarte solo con las mediciones de California

In [ ]:
# Carga del dataset
df = pd.read_csv('pollution_us_2000_2016.csv')

# Filtrar datos de California
df_ca = df[df['State'] == 'California'].copy()

print(f"Registros totales: {len(df)}")
print(f"Registros de California: {len(df_ca)}")
print("\nPrimeras filas del dataset filtrado:")
df_ca.head()

### 2. Limpieza inicial de datos

2. Realiza una limpieza inicial de los datos:
   - Identifica y maneja valores nulos
   - Convierte las columnas de fecha al formato correcto
   - Verifica y corrige valores atípicos

In [ ]:
# Convertir columna de fecha
df_ca['Date Local'] = pd.to_datetime(df_ca['Date Local'])

# Lista de columnas de contaminantes
contaminantes = ['NO2 Mean', 'SO2 Mean', 'CO Mean', 'O3 Mean']

# Verificar valores nulos
print("Valores nulos en el dataset:")
print(df_ca[contaminantes].isnull().sum())

In [ ]:
# Función para quitar valores atípicos
def quitar_atipicos(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Aplicar limpieza de atípicos a cada contaminante
for cont in contaminantes:
    df_ca = quitar_atipicos(df_ca, cont)
    
print("\nDimensiones del DataFrame:")
print(f"Antes de la limpieza: {len(df[df['State'] == 'California'])} filas")
print(f"Después de la limpieza: {len(df_ca)} filas")

3. Analiza los contaminantes principales (NO2, SO2, CO, O3):
   - Calcula promedios mensuales por ciudad
   - Identifica las 5 ciudades con mayores niveles de cada contaminante
   - Determina si hay patrones estacionales

In [ ]:
# Promedios mensuales por ciudad
promedios_mensuales = df_ca.groupby(['City', pd.Grouper(key='Date Local', freq='ME')])[contaminantes].mean()
print(promedios_mensuales)

# Top 5 ciudades por contaminante
for cont in contaminantes:
    print(f"\nTop 5 ciudades con mayor nivel de {cont}:")
    print(df_ca.groupby('City')[cont].mean().nlargest(5))

# Análisis estacional
df_ca['Month'] = df_ca['Date Local'].dt.month
patrones_estacionales = df_ca.groupby('Month')[contaminantes].mean()

# Visualización de patrones estacionales
plt.figure(figsize=(12, 6))
for cont in contaminantes:
    plt.plot(patrones_estacionales.index, patrones_estacionales[cont], label=cont)
plt.title('Patrones Estacionales de Contaminantes')
plt.xlabel('Mes')
plt.ylabel('Nivel promedio')
plt.legend()
plt.show()

4. Crea nuevas columnas derivadas:
   - Índice de calidad del aire simplificado
   - Clasificación por niveles de riesgo
   - Indicadores de cumplimiento de estándares EPA

In [ ]:
# Calcular índice de calidad del aire simplificado
def calcular_indice_calidad(row):
    # Usando los valores AQI existentes
    aqi_values = [
        row['NO2 AQI'],
        row['SO2 AQI'],
        row['CO AQI'],
        row['O3 AQI']
    ]
    return np.mean([x for x in aqi_values if not np.isnan(x)])

df_ca['indice_calidad'] = df_ca.apply(calcular_indice_calidad, axis=1)

# Clasificación por niveles de riesgo
def clasificar_riesgo(indice):
    if indice <= 50: return 'Bueno'
    elif indice <= 100: return 'Moderado'
    elif indice <= 150: return 'Dañino para grupos sensibles'
    else: return 'Dañino'

df_ca['nivel_riesgo'] = df_ca['indice_calidad'].apply(clasificar_riesgo)

Para verlos de diferentes formas:

In [ ]:
# Configuración de estilo
plt.style.use('default')

# 1. Distribución del índice de calidad por nivel de riesgo
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_ca, x='nivel_riesgo', y='indice_calidad')
plt.title('Distribución del Índice de Calidad del Aire por Nivel de Riesgo')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. Evolución temporal del índice de calidad
plt.figure(figsize=(15, 6))
df_ca.groupby('Date Local')['indice_calidad'].mean().rolling(window=30).mean().plot()
plt.title('Evolución del Índice de Calidad del Aire (Media Móvil 30 días)')
plt.xlabel('Fecha')
plt.ylabel('Índice de Calidad del Aire')
plt.tight_layout()
plt.show()

# 3. Proporción de niveles de riesgo
plt.figure(figsize=(10, 10))
df_ca['nivel_riesgo'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Proporción de Niveles de Riesgo')
plt.ylabel('')
plt.tight_layout()
plt.show()

# 4. Top 10 ciudades con peor calidad del aire
plt.figure(figsize=(12, 6))
top_10_ciudades = df_ca.groupby('City')['indice_calidad'].mean().nlargest(10)
sns.barplot(x=top_10_ciudades.values, y=top_10_ciudades.index)
plt.title('Top 10 Ciudades con Mayor Índice de Contaminación')
plt.xlabel('Índice de Calidad del Aire Promedio')
plt.tight_layout()
plt.show()

# 5. Distribución mensual del índice de calidad
plt.figure(figsize=(12, 6))
df_ca.boxplot(column='indice_calidad', by='Month', figsize=(12, 6))
plt.title('Distribución Mensual del Índice de Calidad del Aire')
plt.xlabel('Mes')
plt.ylabel('Índice de Calidad del Aire')
plt.suptitle('')  # Esto elimina el título automático adicional
plt.tight_layout()
plt.show()

5. Realiza análisis temporal:
   - Calcula tendencias anuales
   - Identifica días críticos (con valores extremos)
   - Genera medias móviles semanales y mensuales

In [ ]:
# Análisis temporal
df_ca['Year'] = df_ca['Date Local'].dt.year
tendencias_anuales = df_ca.groupby('Year')[contaminantes].mean()

# Visualización de tendencias anuales
plt.figure(figsize=(12, 6))
for cont in contaminantes:
    plt.plot(tendencias_anuales.index, tendencias_anuales[cont], label=cont)
plt.title('Tendencias Anuales de Contaminantes')
plt.xlabel('Año')
plt.ylabel('Nivel promedio')
plt.legend()
plt.show()

6. Exporta los resultados:
   - Guarda un resumen por ciudad en CSV
   - Crea un archivo con los días críticos identificados
   - Genera un reporte con las estadísticas principales

In [ ]:
# Exportar resultados
resumen_ciudad = df_ca.groupby('City').agg({
    'NO2 Mean': 'mean',
    'SO2 Mean': 'mean',
    'CO Mean': 'mean',
    'O3 Mean': 'mean',
    'indice_calidad': 'mean'
}).round(2)

# Guardar resultados
resumen_ciudad.to_csv('resumen_por_ciudad.csv')

# Días críticos (usando el índice de calidad del aire)
dias_criticos = df_ca[df_ca['indice_calidad'] > df_ca['indice_calidad'].quantile(0.95)]
dias_criticos.to_csv('dias_criticos.csv')

# Matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(df_ca[contaminantes].corr(), annot=True, cmap='coolwarm')
plt.title('Correlación entre Contaminantes')
plt.show()

## Ejercicio 2: Análisis de Datos del Mundial de Fútbol

Utilizando el dataset histórico de la FIFA World Cup disponible en:
https://www.kaggle.com/datasets/abecklas/fifa-world-cup

Desarrolla:
1. Carga y combina los datasets de partidos y equipos usando merge
2. Calcula estadísticas por país:
   - Total de participaciones en mundiales
   - Goles anotados y recibidos
   - Victorias, derrotas y empates
3. Identifica los 5 países más exitosos basándote en una métrica que combines
4. Crea un DataFrame pivotado que muestre el progreso de cada país por año
5. Genera visualizaciones para mostrar las tendencias históricas

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('classic')
sns.set_theme()

In [ ]:
# 1. Cargar los datasets
matches = pd.read_csv('WorldCupMatches.csv')
cups = pd.read_csv('WorldCups.csv')

# Mostrar las primeras filas de cada dataset
print("Dataset de partidos:")
print(matches.head())
print("\nDataset de copas mundiales:")
print(cups.head())

In [ ]:
# Preparación de datos - Crear registros separados para equipos local y visitante
home_team = matches[['Year', 'Home Team Name', 'Home Team Goals', 'Away Team Goals']].copy()
away_team = matches[['Year', 'Away Team Name', 'Home Team Goals', 'Away Team Goals']].copy()

# Renombrar columnas
home_team.columns = ['Year', 'Team', 'Goals_For', 'Goals_Against']
away_team.columns = ['Year', 'Team', 'Goals_Against', 'Goals_For']

# Combinar los datos
all_games = pd.concat([home_team, away_team])
all_games.head()

In [ ]:
# 2. Calcular estadísticas por país

# Total de participaciones en mundiales
participaciones = all_games.groupby('Team')['Year'].nunique().sort_values(ascending=False)
participaciones.head()

In [ ]:
# Goles anotados y recibidos
goles = all_games.groupby('Team').agg({
    'Goals_For': 'sum',
    'Goals_Against': 'sum'
}).round(2)
goles.head()

In [ ]:
# Calcular victorias, derrotas y empates
all_games['Result'] = np.where(all_games['Goals_For'] > all_games['Goals_Against'], 'Win',
                              np.where(all_games['Goals_For'] < all_games['Goals_Against'], 'Loss', 'Draw'))
resultados = all_games.groupby('Team')['Result'].value_counts().unstack(fill_value=0)
resultados.head()

In [ ]:
# Combinar todas las estadísticas
stats = pd.concat([participaciones, goles, resultados], axis=1).fillna(0)
stats.columns = ['Participaciones', 'Goles_Favor', 'Goles_Contra', 'Derrotas', 'Empates', 'Victorias']

print("Estadísticas por país:")
print(stats.head(10))

In [ ]:
# 3. Identificar los 5 países más exitosos

# Crear métrica compuesta: (3*Victorias + Empates + 2*Goles_Favor - Goles_Contra + 5*Participaciones)/Participaciones
stats['Score'] = (3*stats['Victorias'] + stats['Empates'] + 2*stats['Goles_Favor'] - 
                  stats['Goles_Contra'] + 5*stats['Participaciones'])/stats['Participaciones']

top_5 = stats.nlargest(5, 'Score')
print("Top 5 países más exitosos:")
print(top_5[['Score', 'Victorias', 'Empates', 'Derrotas', 'Goles_Favor', 'Goles_Contra']])

In [ ]:
# 4. DataFrame pivotado para progreso por año
print(all_games.head(20))

progreso_anual = pd.pivot_table(all_games,
                               values='Goals_For',
                               index='Year',
                               columns='Team',
                               aggfunc='sum')

print("Progreso de goles por año y país:")
print(progreso_anual.head())

In [ ]:
# 5. Visualizaciones

# Gráfico de barras para el Top 5
plt.figure(figsize=(12, 6))
sns.barplot(data=top_5.reset_index(), x='Team', y='Score')
plt.title('Top 5 Países más Exitosos en la Historia de los Mundiales')
plt.xticks(rotation=45)
plt.show()

# Evolución de goles para el Top 5
plt.figure(figsize=(15, 8))
for team in top_5.index:
    plt.plot(progreso_anual.index, progreso_anual[team].fillna(0), label=team, marker='o')
plt.title('Evolución de Goles por Mundial - Top 5 Países')
plt.xlabel('Año')
plt.ylabel('Goles Anotados')
plt.legend()
plt.grid(True)
plt.show()

# Gráfico de radar para comparar métricas del Top 5
metrics = ['Victorias', 'Empates', 'Goles_Favor', 'Participaciones']
angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False)

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

for idx, team in enumerate(top_5.index):
    values = top_5.loc[team, metrics].values
    values = np.concatenate((values, [values[0]]))
    angles_plot = np.concatenate((angles, [angles[0]]))
    ax.plot(angles_plot, values, 'o-', linewidth=2, label=team)
    ax.fill(angles_plot, values, alpha=0.25)

ax.set_xticks(angles)
ax.set_xticklabels(metrics)
plt.title('Comparación de Métricas - Top 5 Países')
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
plt.show()

### Análisis de Resultados:

1. **Participación histórica:**
   - Brasil, Alemania e Italia son los países con mayor número de participaciones
   - Estos países también lideran en victorias y goles anotados

2. **Eficiencia ofensiva:**
   - Brasil destaca por su alta proporción de goles anotados vs. recibidos
   - Alemania muestra una gran consistencia en todas las métricas

3. **Evolución histórica:**
   - Se observa una tendencia al alza en el número de goles por mundial
   - Los equipos top muestran mayor regularidad en sus performances

4. **Métricas compuestas:**
   - La métrica creada pondera diferentes aspectos del éxito
   - Los resultados coinciden con la percepción histórica del éxito de estos equipos

5. **Tendencias actuales:**
   - Mayor paridad entre equipos en mundiales recientes
   - Menor dominancia de equipos tradicionales en últimas ediciones

## Ejercicio 3: Análisis de Series Temporales de Bolsa

Usando el dataset de Yahoo Finance para el índice S&P 500 (^GSPC)

In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta

# Configuración básica de visualización
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 6]

# Para evitar warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. Obtener datos de los últimos 5 años
# Calculamos las fechas
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

# Descargamos los datos usando yfinance directamente
ticker = yf.Ticker('^GSPC')
df = ticker.history(start=start_date, end=end_date)

print("Primeras 5 filas del dataset:")
df.head()

In [ ]:
# 2. Calcular retornos

# Retornos diarios
df['daily_return'] = df['Close'].pct_change()

# Retornos semanales
df['weekly_return'] = df['Close'].pct_change(periods=5)

# Retornos mensuales (aproximadamente 21 días de trading)
df['monthly_return'] = df['Close'].pct_change(periods=21)

print("Estadísticas descriptivas de los retornos:")
df[['daily_return', 'weekly_return', 'monthly_return']].describe()

In [ ]:
# 3. Identificar los 10 días con mayor volatilidad
volatility_days = df['daily_return'].abs().sort_values(ascending=False).head(10)

print("Los 10 días con mayor volatilidad:")
print(volatility_days)

# Visualización de la volatilidad
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['daily_return'].abs(), alpha=0.5)
plt.title('Volatilidad Diaria del S&P 500')
plt.ylabel('Volatilidad (|Retorno Diario|)')
plt.show()

In [ ]:
# 4. Implementar ventana móvil de 20 días

# Media móvil
df['MA20'] = df['Close'].rolling(window=20).mean()

# Desviación estándar móvil
df['STD20'] = df['Close'].rolling(window=20).std()

# Máximos y mínimos móviles
df['MAX20'] = df['Close'].rolling(window=20).max()
df['MIN20'] = df['Close'].rolling(window=20).min()

# Visualización
plt.figure(figsize=(15, 8))
plt.plot(df.index, df['Close'], label='Precio', alpha=0.7)
plt.plot(df.index, df['MA20'], label='Media Móvil 20 días', linewidth=2)
plt.fill_between(df.index, 
                 df['MA20'] - 2*df['STD20'], 
                 df['MA20'] + 2*df['STD20'], 
                 alpha=0.2, 
                 label='Bandas de Volatilidad')
plt.title('S&P 500 con Media Móvil y Bandas de Volatilidad')
plt.legend()
plt.show()

In [ ]:
# 5. Crear indicadores técnicos

def calculate_rsi(data, periods=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Calcular RSI
df['RSI'] = calculate_rsi(df['Close'])

# Calcular MACD
exp1 = df['Close'].ewm(span=12, adjust=False).mean()
exp2 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = exp1 - exp2
df['Signal Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

In [ ]:
# 6. Visualizar indicadores
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12))

# Precio y Media Móvil
ax1.plot(df.index, df['Close'], label='Precio')
ax1.plot(df.index, df['MA20'], label='MA20')
ax1.set_title('S&P 500 con Media Móvil de 20 días')
ax1.legend()

# RSI
ax2.plot(df.index, df['RSI'])
ax2.axhline(y=70, color='r', linestyle='--')
ax2.axhline(y=30, color='g', linestyle='--')
ax2.set_title('RSI (14 períodos)')

# MACD
ax3.plot(df.index, df['MACD'], label='MACD')
ax3.plot(df.index, df['Signal Line'], label='Señal')
ax3.set_title('MACD')
ax3.legend()

plt.tight_layout()
plt.show()

### Conclusiones:

1. Hemos analizado los datos del S&P 500 de los últimos 5 años, calculando diferentes tipos de retornos (diarios, semanales y mensuales).

2. Identificamos los días con mayor volatilidad, lo que nos permite detectar eventos significativos en el mercado.

3. Implementamos indicadores técnicos populares:
   - Media móvil de 20 días para identificar tendencias
   - RSI para identificar condiciones de sobrecompra/sobreventa
   - MACD para identificar cambios en la tendencia y momentum

4. Las visualizaciones nos permiten ver claramente:
   - La tendencia general del mercado
   - Períodos de alta volatilidad
   - Señales de compra/venta según los indicadores técnicos

## Ejercicio 4: Análisis de Datos de COVID-19

Utilizando el dataset de Our World in Data sobre COVID-19:
https://github.com/owid/covid-19-data/tree/master/public/data

Desarrolla:
1. Carga y limpia el dataset
2. Calcula para cada país:
   - Tasa de positividad diaria
   - Media móvil de 7 días de casos nuevos
   - Tiempo hasta alcanzar picos de casos
3. Agrupa países por continente y compara métricas clave
4. Identifica correlaciones entre variables (casos, muertes, vacunación)
5. Crea un dashboard básico con las métricas más relevantes

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Configuración de visualización
plt.style.use('default')
pd.set_option('display.max_columns', None)

In [ ]:
# 1. Carga y limpieza del dataset
url = 'owid-covid-data.csv'
df = pd.read_csv(url)
print(df.head())

# Convertimos la columna de fecha a datetime
df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
# Eliminamos columnas con más del 50% de valores nulos
null_thresh = len(df) * 0.5
df = df.dropna(axis=1, thresh=null_thresh)

# Rellenamos valores nulos restantes
df = df.fillna({
    'new_cases': 0,
    'new_deaths': 0,
    'total_cases': df.groupby('location')['total_cases'].ffill(),
    'total_deaths': df.groupby('location')['total_deaths'].ffill()
})

print('Dimensiones del dataset:', df.shape)
print('\nColumnas disponibles:\n', df.columns.tolist())

In [ ]:
# 2. Cálculos por país

# Verificamos las columnas disponibles
print("Columnas disponibles en el dataset:")
print(df.columns.tolist())

# Media móvil de 7 días de casos nuevos
df['cases_7day_avg'] = df.groupby('location')['new_cases'].rolling(window=7, center=True).mean().reset_index(0, drop=True)

# Intentamos calcular la tasa de positividad si los datos están disponibles
if 'new_tests' in df.columns:
    df['positivity_rate'] = df['new_cases'] / df['new_tests'] * 100
else:
    print("\nNota: No se puede calcular la tasa de positividad porque 'new_tests' no está disponible en el dataset")
    df['positivity_rate'] = None

# Tiempo hasta alcanzar picos de casos
def find_peak_day(group):
    peak_cases = group['new_cases'].max()
    peak_date = group[group['new_cases'] == peak_cases]['date'].iloc[0]
    first_date = group['date'].min()
    return (peak_date - first_date).days

peak_days = df.groupby('location').apply(find_peak_day)
peak_days = pd.DataFrame(peak_days, columns=['days_to_peak'])

# Mostramos los resultados para algunos países
sample_countries = ['United States', 'Spain', 'Italy', 'Brazil', 'India']
print('\nDías hasta alcanzar el pico de casos:')
print(peak_days.loc[sample_countries])

In [ ]:
# 3. Análisis por continente

# Creamos métricas agregadas por continente
continent_metrics = df.groupby('continent').agg({
    'total_cases': 'max',
    'total_deaths': 'max',
    'new_cases': 'mean',
    'new_deaths': 'mean'
}).round(2)

# Visualizamos las métricas por continente
plt.figure(figsize=(12, 6))
continent_metrics['total_cases'].plot(kind='bar')
plt.title('Total de Casos por Continente')
plt.xticks(rotation=45)
plt.ylabel('Número de Casos')
plt.tight_layout()
plt.show()

print('\nMétricas por continente:')
print(continent_metrics)

In [ ]:
# 4. Análisis de correlaciones

# Seleccionamos variables numéricas disponibles
numeric_columns = df.select_dtypes(include=[np.number]).columns
correlation_vars = [col for col in [
    'new_cases', 'new_deaths', 'total_cases', 'total_deaths',
    'people_vaccinated', 'people_fully_vaccinated'
] if col in numeric_columns]

# Calculamos la matriz de correlación
correlation_matrix = df[correlation_vars].corr()

# Visualizamos la matriz de correlación
plt.figure(figsize=(10, 8))
im = plt.imshow(correlation_matrix, cmap='coolwarm', aspect='auto')
plt.colorbar(im)

# Añadimos las etiquetas
plt.xticks(range(len(correlation_vars)), correlation_vars, rotation=45, ha='right')
plt.yticks(range(len(correlation_vars)), correlation_vars)

# Añadimos los valores numéricos
for i in range(len(correlation_vars)):
    for j in range(len(correlation_vars)):
        text = plt.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                       ha='center', va='center')

plt.title('Matriz de Correlación de Variables COVID-19')
plt.tight_layout()
plt.show()

In [ ]:
# 5. Dashboard básico

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Evolución temporal de casos nuevos (top 5 países)
top_countries = df.groupby('location')['new_cases'].sum().nlargest(5).index
for country in top_countries:
    country_data = df[df['location'] == country]
    axes[0, 0].plot(country_data['date'], country_data['cases_7day_avg'], label=country)
axes[0, 0].set_title('Media Móvil 7 días - Casos Nuevos')
axes[0, 0].legend()
axes[0, 0].tick_params(axis='x', rotation=45)

# Gráfico 2: Casos totales por continente
continent_cases = df.groupby('continent')['total_cases'].max()
axes[0, 1].bar(continent_cases.index, continent_cases.values)
axes[0, 1].set_title('Casos Totales por Continente')
axes[0, 1].tick_params(axis='x', rotation=45)

# Gráfico 3: Relación entre casos y muertes
axes[1, 0].scatter(df['total_cases'], df['total_deaths'], alpha=0.5)
axes[1, 0].set_title('Relación entre Casos Totales y Muertes')
axes[1, 0].set_xlabel('Casos Totales')
axes[1, 0].set_ylabel('Muertes Totales')

# Gráfico 4: Distribución de días hasta el pico
axes[1, 1].hist(peak_days['days_to_peak'], bins=30)
axes[1, 1].set_title('Distribución de Días hasta el Pico de Casos')
axes[1, 1].set_xlabel('Días hasta el pico')
axes[1, 1].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

### Conclusiones:

1. La limpieza de datos reveló que algunas variables tenían una cantidad significativa de valores faltantes, especialmente en las métricas de pruebas y vacunación.

2. La eliminación de columnas con más del 50% de valores nulos nos ayudó a trabajar con un conjunto de datos más manejable y significativo.

3. Se observa una fuerte correlación entre casos totales y muertes totales, aunque la relación no es perfectamente lineal.

4. Los países alcanzaron sus picos de casos en diferentes momentos, lo que refleja la naturaleza asincrónica de la pandemia.

5. El análisis por continente muestra diferencias significativas en el impacto de la pandemia, tanto en casos totales como en mortalidad.

6. La media móvil de 7 días ayuda a visualizar mejor las tendencias al suavizar las fluctuaciones diarias en los datos.

## Ejercicio 5: Análisis de Reseñas de Amazon

Usando el dataset de reseñas de Amazon disponible en:
https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

Realiza:
1. Carga y preprocesa el dataset
2. Analiza la distribución de puntuaciones
3. Identifica patrones temporales en las reseñas:
   - Evolución de puntuaciones promedio por mes/año
   - Cambios en la longitud de las reseñas
4. Agrupa productos por categoría y analiza diferencias en:
   - Puntuación promedio
   - Cantidad de reseñas
   - Sentimiento general
5. Crea un sistema simple de detección de reseñas potencialmente falsas basado en múltiples criterios

Conceptos evaluados: text processing, análisis temporal, detección de anomalías, agregaciones complejas

### 1. Carga y preprocesamiento de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

# Configuración de visualización
sns.set_theme()  # Usamos el tema por defecto de seaborn en lugar de plt.style
pd.set_option('display.max_columns', None)

# Cargar el dataset
df = pd.read_csv('Reviews.csv')

# Mostrar información básica del dataset
print("Información del dataset:")
print(df.info())

# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Limpieza básica
df['Time'] = pd.to_datetime(df['Time'], unit='s')
df['ReviewLength'] = df['Text'].str.len()
df['Year'] = df['Time'].dt.year
df['Month'] = df['Time'].dt.month

print("\nPrimeras 5 filas del dataset procesado:")
df.head()

### 2. Análisis de distribución de puntuaciones

In [ ]:
# Crear gráfico de distribución de puntuaciones
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Score')
plt.title('Distribución de Puntuaciones')
plt.xlabel('Puntuación')
plt.ylabel('Cantidad de Reseñas')
plt.show()

# Estadísticas descriptivas de las puntuaciones
print("Estadísticas descriptivas de las puntuaciones:")
print(df['Score'].describe())

# Calcular porcentajes por puntuación
score_percentages = (df['Score'].value_counts() / len(df) * 100).round(2)
print("\nPorcentaje de reseñas por puntuación:")
print(score_percentages.sort_index())

### 3. Patrones temporales en las reseñas

In [ ]:
# Evolución de puntuaciones promedio por mes/año
monthly_scores = df.groupby(['Year', 'Month'])['Score'].mean().reset_index()
monthly_scores['Date'] = pd.to_datetime(monthly_scores[['Year', 'Month']].assign(DAY=1))

plt.figure(figsize=(12, 6))
plt.plot(monthly_scores['Date'], monthly_scores['Score'])
plt.title('Evolución de Puntuaciones Promedio por Mes')
plt.xlabel('Fecha')
plt.ylabel('Puntuación Promedio')
plt.grid(True)
plt.show()

# Cambios en la longitud de las reseñas
monthly_lengths = df.groupby(['Year', 'Month'])['ReviewLength'].mean().reset_index()
monthly_lengths['Date'] = pd.to_datetime(monthly_lengths[['Year', 'Month']].assign(DAY=1))

plt.figure(figsize=(12, 6))
plt.plot(monthly_lengths['Date'], monthly_lengths['ReviewLength'])
plt.title('Evolución de la Longitud Promedio de Reseñas por Mes')
plt.xlabel('Fecha')
plt.ylabel('Longitud Promedio')
plt.grid(True)
plt.show()

### 4. Análisis por categoría de producto

In [ ]:
# Crear un análisis básico por ProductId
product_analysis = df.groupby('ProductId').agg({
'Score': ['mean', 'count'],
'ReviewLength': 'mean'
}).round(2)

product_analysis.columns = ['Puntuación_Promedio', 'Cantidad_Reseñas', 'Longitud_Promedio']
product_analysis = product_analysis.reset_index()

# Mostrar productos más reseñados
print("Top 10 productos más reseñados:")
print(product_analysis.nlargest(10, 'Cantidad_Reseñas'))

# Visualizar relación entre cantidad de reseñas y puntuación promedio
plt.figure(figsize=(10, 6))
plt.scatter(product_analysis['Cantidad_Reseñas'], 
           product_analysis['Puntuación_Promedio'], 
           alpha=0.5)
plt.title('Relación entre Cantidad de Reseñas y Puntuación Promedio')
plt.xlabel('Cantidad de Reseñas')
plt.ylabel('Puntuación Promedio')
plt.show()

### 5. Sistema de detección de reseñas potencialmente falsas

In [ ]:
def detect_suspicious_reviews(df):
    # Crear características para detección
    df['suspicious_score'] = 0
    
    # Crear columna de fecha
    df['Date'] = df['Time'].dt.date  # Primero extraemos la fecha
    
    # 1. Reseñas extremadamente cortas o largas
    length_quantiles = df['ReviewLength'].quantile([0.05, 0.95])
    df.loc[df['ReviewLength'] < length_quantiles[0.05], 'suspicious_score'] += 1
    df.loc[df['ReviewLength'] > length_quantiles[0.95], 'suspicious_score'] += 1

    # 2. Usuario con muchas reseñas en poco tiempo
    user_daily_reviews = df.groupby(['UserId', 'Date']).size().reset_index(name='daily_reviews')
    suspicious_users = user_daily_reviews[user_daily_reviews['daily_reviews'] > 5]['UserId'].unique()
    df.loc[df['UserId'].isin(suspicious_users), 'suspicious_score'] += 1

    # 3. Puntuaciones extremas (1 o 5) con texto corto
    df.loc[(df['Score'].isin([1, 5])) & (df['ReviewLength'] < 50), 'suspicious_score'] += 1

    # Clasificar reseñas como sospechosas si tienen un score alto
    df['is_suspicious'] = df['suspicious_score'] >= 2

    return df

# Aplicar el sistema de detección
df = detect_suspicious_reviews(df)

# Mostrar resultados
print("Porcentaje de reseñas sospechosas:")
print(f"{(df['is_suspicious'].mean() * 100).round(2)}%")

# Analizar características de reseñas sospechosas vs no sospechosas
suspicious_stats = df.groupby('is_suspicious').agg({
    'Score': 'mean',
    'ReviewLength': 'mean',
    'ProductId': 'count'
}).round(2)

print("\nEstadísticas comparativas:")
print(suspicious_stats)